# Tweet Scraper

Este es el notebook que usaremos para el scrapeo de datos


## Librerias

Para el scrapeo usaremos las siguientes librerias:
* Sntwitter
* Pandas

In [1]:
import datetime
from datetime import timedelta

import snscrape.modules.twitter as snstwitter

import pandas as pd
import seaborn as sns

from tqdm import tqdm

from JATS.src.JATS import JATS

## Parametros



In [2]:
query = '"BTC" OR "bitcoin"'

date_from = datetime.date(2018, 3, 2)
date_until = datetime.date(2018, 3, 3)

tweet_list = JATS.get_tweets(query, date_from, date_until, verbose = True)

Day 2018-03-02
0  /  -1
2500  /  -1
5000  /  -1
7500  /  -1
10000  /  -1
12500  /  -1
15000  /  -1
17500  /  -1
20000  /  -1
22500  /  -1
25000  /  -1
27500  /  -1
30000  /  -1
32500  /  -1
35000  /  -1
37500  /  -1
40000  /  -1
42500  /  -1


### Lectura de ficheros ya existentes

In [ ]:
tweet_df = pd.read_csv(file_name, sep=',')

tweet_df["Datetime"] = pd.to_datetime(tweet_df["Datetime"])

## Creación de nuevas columnas del DataFrame

In [ ]:
tweet_df['sentiment'] = ""
tweet_df['round_time'] = ""

tweet_df["sentiment"] = pd.to_numeric(tweet_df["sentiment"])
tweet_df["round_time"] = pd.to_datetime(tweet_df["round_time"])

## Analisis de sentimientos

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

### Preparación del Analisis

En caso de ser nuestra primera ejecución, deberemos instalar un conjunto de datasets utiles para *nltk*.

In [ ]:
nltk.download([     
    "names",
    "stopwords",
    "state_union",
    "twitter_samples",
    "movie_reviews",
    "averaged_perceptron_tagger",
    "vader_lexicon",
    "punkt",
])

### Limpieza de dataset

In [ ]:
import re
import emoji

def clean_tweet(tweet):
    tweet = re.sub("@[A-Za-z0-9_]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = emoji.get_emoji_regexp().sub(r'', tweet)
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    return tweet

In [ ]:

def get_round_time(data):
    return data.replace(second=0, microsecond=0, minute=0)

def get_sentiment(text):
    """
        Cleans and analyzes text.
        
        Parameters:
        text(string): Bunch of text
        
        Returns:
        Compound sentiment from NLTK polarity
    
    """
    
    clean_text = clean_tweet(text)
    polarity = sia.polarity_scores(clean_text)
    return polarity['compound']

def prepare_data(tweet_df, verbose=False):
    tweet_df['round_time'] = tweet_df['Datetime'].round('h')
    
    for index, row in tweet_df.iterrows():
        tweet_df.loc[i, 'sentiment'] = get_sentiment(tweet_df['Text'].iloc[i])
            

In [ ]:
prepare_data(tweet_df)

## Analisis de Sentimiento

Eliminaremos los valores nulos ya que parece que cuando el algoritmo no es capaz de determinar el sentimiento, tiende a ponerle un 0, creando una desviación del sentimiento real.

Lo primero que haremos será mostrar la **media del sentimiento** y una **gráfica de distribución del sentimiento**.

In [ ]:
average_sentiment = tweet_df[tweet_df['sentiment'] != 0]['sentiment'].mean()

print(f"Average sentiment: {average_sentiment}")

sns.distplot(tweet_df[tweet_df['sentiment'] != 0]['sentiment'])